In [4]:
# 导入环境
import os, sys
current_path = os.path.abspath(os.path.dirname('.'))  # 获取文件目录
project_path = current_path[:current_path.find('RIGEL') + len('RIGEL')]  # 获取项目根路径，内容为当前项目的名字，即RIGEL
sys.path.append(project_path)  # 将项目根路径添加到系统路径中，以便导入项目中的模块

import numpy as np
from src import Dynamics, Activation, ESN, VISION, Dataset_makeup
import matplotlib.pyplot as plt